# Face and Eyeball Detection

## Detection on a Image

In [1]:
import numpy as np
import cv2
#cascPath = sys.argv[1]
#faceCascade = cv2.CascadeClassifier(cascPath)
spath = 'C:\\ProgramData\\Anaconda3\\Lib\\site-packages\\cv2\\data\\'

#eye_cascade = cv2.CascadeClassifier('haarcascade_eye.xml')
face_cascade = cv2.CascadeClassifier(spath+'haarcascade_frontalface_alt.xml')
eye_cascade = cv2.CascadeClassifier(spath+'haarcascade_eye_tree_eyeglasses.xml')

img = cv2.imread("C:\\Users\\Puja Kumari\\Downloads\\p.jpg")
#print(np.shape(img))
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
#print(np.shape(gray))

#TRYING TO SHOW MULTIPLE IMAGES IN ONE WINDOW
#converting image to have 3 channels
grey = cv2.cvtColor(gray, cv2.COLOR_GRAY2BGR)
stack = np.hstack((img, grey))

faces = face_cascade.detectMultiScale(gray, 1.3, 5)
for(x,y,w,h) in faces:
    img = cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),2)
    roi_gray = gray[y:y+h, x:x+w]
    roi_color = img[y:y+h, x:x+w]
    eyes = eye_cascade.detectMultiScale(roi_gray)
    for(ex,ey,ew,eh) in eyes:
        cv2.rectangle(roi_color,(ex,ey),(ex+ew,ey+eh),(0,255,0),2)

cv2.imshow('IMAGE', img)
cv2.waitKey(0)
cv2.destroyAllWindows()

KeyboardInterrupt: 

### Detection on an image from the camera feed

In [ ]:
import numpy as np
import cv2
spath = 'C:\\ProgramData\\Anaconda3\\Lib\\site-packages\\cv2\\data\\'
#face_cascade = cv2.CascadeClassifier(r'C:\Users\Lenovo\AppData\Local\Programs\Python\Python37-32\\Lib\\site-packages\\cv2\\data\\haarcascade_frontalface_alt.xml')
#eye_cascade = cv2.CascadeClassifier(r'C:\Users\Lenovo\AppData\Local\Programs\Python\Python37-32\\Lib\\site-packages\\cv2\\data\\haarcascade_eye_tree_eyeglasses.xml')
face_cascade = cv2.CascadeClassifier(spath+'haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier(spath+'haarcascade_eye.xml')
cv2.VideoCapture(0).release()
cap = cv2.VideoCapture(0)

while 1:
    ret, img = cap.read()
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    
    faces = face_cascade.detectMultiScale(
        gray, 
        scaleFactor = 1.1, 
        minNeighbors = 5,
        minSize = (30, 30),
        flags = cv2.CASCADE_SCALE_IMAGE
    )

    for (x,y,w,h) in faces:
        cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),2)
        roi_gray = gray[y:y+h, x:x+w]
        roi_color = img[y:y+h, x:x+w]
        
        eyes = eye_cascade.detectMultiScale(roi_gray)
        for (ex,ey,ew,eh) in eyes:
            cv2.rectangle(roi_color,(ex,ey),(ex+ew,ey+eh),(0,255,0),2)
            

    cv2.imshow('img',img)
    k = cv2.waitKey(1) & 0xff
    if k == 27:
        break

        
cap.release()
cv2.destroyAllWindows()

## Sample Code to Detect the Pupil on an image  (~Understood)

In [ ]:
#Identify pupils. Based on beta 1

import numpy as np
import cv2
import time
spath = 'C:\\ProgramData\\Anaconda3\\Lib\\site-packages\\cv2\\data\\'
cap = cv2.VideoCapture(0)     #640,480
w = 640
h = 480

while(cap.isOpened()):
    ret, frame = cap.read()
    if ret==True:
    
        #downsample
        #frameD = cv2.pyrDown(cv2.pyrDown(frame))
        #frameDBW = cv2.cvtColor(frameD,cv2.COLOR_RGB2GRAY)
    
        #detect face
        frame = cv2.cvtColor(frame,cv2.COLOR_RGB2GRAY)
        #faces = cv2.CascadeClassifier(r'C:\Users\Lenovo\AppData\Local\Programs\Python\Python37-32\\Lib\\site-packages\\cv2\\data\\haarcascade_eye_tree_eyeglasses.xml')
        
    
        faces = cv2.CascadeClassifier(spath+'haarcascade_frontalface_default.xml')
        detected = faces.detectMultiScale(frame, 1.3, 5)
        #detected2 = faces.detectMultiScale(frameDBW, 1.3, 5)
        
        pupilFrame = frame
        pupilO = frame
        windowClose = np.ones((5,5),np.uint8)
        windowOpen = np.ones((2,2),np.uint8)
        windowErode = np.ones((2,2),np.uint8)

        #draw square
        for (x,y,w,h) in detected:
            cv2.rectangle(frame, (x,y), ((x+w),(y+h)), (0,0,255),1)    
            cv2.line(frame, (x,y), ((x+w,y+h)), (0,0,255),1)
            cv2.line(frame, (x+w,y), ((x,y+h)), (0,0,255),1)
            pupilFrame = cv2.equalizeHist(frame[int(y+(h*.25)):(y+h), x:(x+w)])
            pupilO = pupilFrame
            ret, pupilFrame = cv2.threshold(pupilFrame,70,255,cv2.THRESH_BINARY)        #50 ..nothin 70 is better
            pupilFrame = cv2.morphologyEx(pupilFrame, cv2.MORPH_CLOSE, windowClose)
            pupilFrame = cv2.morphologyEx(pupilFrame, cv2.MORPH_ERODE, windowErode)
            pupilFrame = cv2.morphologyEx(pupilFrame, cv2.MORPH_OPEN, windowOpen)

            #so above we do image processing to get the pupil..
            #now we find the biggest blob and get the centriod
            
            threshold = cv2.inRange(pupilFrame,250,255)        #get the blobs
            contours, hierarchy = cv2.findContours(threshold,cv2.RETR_LIST,cv2.CHAIN_APPROX_SIMPLE)
            
            #if there are 3 or more blobs, delete the biggest and delete the left most for the right eye
            #if there are 2 blob, take the second largest
            #if there are 1 or less blobs, do nothing
            
            if len(contours) >= 2:
                #find biggest blob
                maxArea = 0
                MAindex = 0            #to get the unwanted frame 
                distanceX = []        #delete the left most (for right eye)
                currentIndex = 0 
                for cnt in contours:
                    area = cv2.contourArea(cnt)
                    center = cv2.moments(cnt)
                    if center['m00'] != 0:
                        cx = int(center["m10"] / center["m00"])
                        cy = int(center["m01"] / center["m00"])
                    else:
                        cx,cy = 0, 0
                    distanceX.append(cx)    
                    if area > maxArea:
                        maxArea = area
                        MAindex = currentIndex
                    currentIndex = currentIndex + 1
        
                del contours[MAindex]        #remove the picture frame contour
                del distanceX[MAindex]
            
            eye = 'right'

            if len(contours) >= 2:        #delete the left most blob for right eye
                if eye == 'right':
                    edgeOfEye = distanceX.index(min(distanceX))
                else:
                    edgeOfEye = distanceX.index(max(distanceX))    
                del contours[edgeOfEye]
                del distanceX[edgeOfEye]

            if len(contours) >= 1:        #get largest blob
                maxArea = 0
                for cnt in contours:
                    area = cv2.contourArea(cnt)
                    if area > maxArea:
                        maxArea = area
                        largeBlob = cnt
                    
            if len(largeBlob) > 0:    
                center = cv2.moments(largeBlob)
                cx,cy = int(center['m10']/center['m00']), int(center['m01']/center['m00'])
                cv2.circle(pupilO,(cx,cy),5,255,-1)

    
        #show picture
        cv2.imshow('frame',pupilO)
        cv2.imshow('frame2',pupilFrame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    #else:
        #break

# Release everything if job is finished
cap.release()
cv2.destroyAllWindows()


## Detection of Pupil in an image

In [ ]:
import numpy as np
import cv2
spath = 'C:\\ProgramData\\Anaconda3\\Lib\\site-packages\\cv2\\data\\'
face_cascade = cv2.CascadeClassifier(spath+'haarcascade_frontalface_alt.xml')
eye_cascade = cv2.CascadeClassifier(spath+'haarcascade_eye_tree_eyeglasses.xml')

img = cv2.imread('C:\\Users\\Puja Kumari\\Downloads\\p.jpg')
#print(np.shape(img))
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
#print(np.shape(gray))

#TRYING TO SHOW MULTIPLE IMAGES IN ONE WINDOW
#converting image to have 3 channels
grey = cv2.cvtColor(gray, cv2.COLOR_GRAY2BGR)
stack = np.hstack((img, grey))

faces = face_cascade.detectMultiScale(gray, 1.3, 5)
for(x,y,w,h) in faces:
    img = cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),2)
    roi_gray = gray[y:y+h, x:x+w]
    roi_color = img[y:y+h, x:x+w]
    eyes = eye_cascade.detectMultiScale(roi_gray)
    
    ########################A#D#A#P#T#I#N#G#T#O#N#E#W#C#O#D#E############################################
    pupilFrame = roi_gray
    pupilO = roi_gray
    windowClose = np.ones((5,5),np.uint8)
    windowOpen = np.ones((2,2),np.uint8)
    windowErode = np.ones((2,2),np.uint8)
    #####################################################################################################
    for(ex,ey,ew,eh) in eyes:
        cv2.rectangle(roi_color,(ex,ey),(ex+ew,ey+eh),(0,255,0),2)
        cv2.line(roi_color, (ex,ey), ((ex+ew,ey+eh)), (0,0,255),1)
        cv2.line(roi_color, (ex+ew,ey), ((ex,ey+eh)), (0,0,255),1)
        pupilFrame = cv2.equalizeHist(roi_gray[int(ey+(eh*.25)):(ey+eh), ex:(ex+ew)])
        pupilO = pupilFrame
        ret, pupilFrame = cv2.threshold(pupilFrame,50,255,cv2.THRESH_BINARY)        #50 ..nothin 70 is better
        pupilFrame = cv2.morphologyEx(pupilFrame, cv2.MORPH_CLOSE, windowClose)
        pupilFrame = cv2.morphologyEx(pupilFrame, cv2.MORPH_ERODE, windowErode)
        pupilFrame = cv2.morphologyEx(pupilFrame, cv2.MORPH_OPEN, windowOpen)
        
        threshold = cv2.inRange(pupilFrame,250,255)        #get the blobs
        
        contours, hierarchy = cv2.findContours(threshold,cv2.RETR_LIST,cv2.CHAIN_APPROX_SIMPLE)
        
        if len(contours) >= 2:
                #find biggest blob
                maxArea = 0
                MAindex = 0            #to get the unwanted frame 
                distanceX = []        #delete the left most (for right eye)
                currentIndex = 0 
                for cnt in contours:
                    area = cv2.contourArea(cnt)
                    center = cv2.moments(cnt)
                    if center['m00'] != 0:
                        cx = int(center["m10"] / center["m00"])
                        cy = int(center["m01"] / center["m00"])
                    else:
                        cx,cy = 0, 0
                    distanceX.append(cx)    
                    if area > maxArea:
                        maxArea = area
                        MAindex = currentIndex
                    currentIndex = currentIndex + 1
        
                del contours[MAindex]        #remove the picture frame contour
                del distanceX[MAindex]
            
        eye = 'right'

        if len(contours) >= 2:        #delete the left most blob for right eye
            if eye == 'right':
                edgeOfEye = distanceX.index(min(distanceX))
            else:
                edgeOfEye = distanceX.index(max(distanceX))    
            del contours[edgeOfEye]
            del distanceX[edgeOfEye]

        if len(contours) >= 1:        #get largest blob
                maxArea = 0
                for cnt in contours:
                    area = cv2.contourArea(cnt)
                    if area > maxArea:
                        maxArea = area
                        largeBlob = cnt
                    
        if len(largeBlob) > 0:    
                center = cv2.moments(largeBlob)
                cx,cy = int(center['m10']/center['m00']), int(center['m01']/center['m00'])
                cv2.circle(pupilO,(cx,cy),5,255,-1)


cv2.imshow('frame',pupilO)
#cv2.imshow('frame2',pupilFrame)
cv2.imshow('IMAGE', img)
cv2.waitKey(0)
cv2.destroyAllWindows()

## Detection of Pupil from camera feed

In [ ]:
import numpy as np
import cv2
spath = 'C:\\ProgramData\\Anaconda3\\Lib\\site-packages\\cv2\\data\\'
face_cascade = cv2.CascadeClassifier(spath+'haarcascade_frontalface_alt.xml')
eye_cascade = cv2.CascadeClassifier(spath+'haarcascade_eye_tree_eyeglasses.xml')

cap = cv2.VideoCapture(0)


while True:
    ret, img = cap.read()
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    faces = face_cascade.detectMultiScale(
        gray, 
        scaleFactor = 1.1, 
        minNeighbors = 5,
        minSize = (30, 30),
        flags = cv2.CASCADE_SCALE_IMAGE
    )

    for (x,y,w,h) in faces:
        cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),2)
        roi_gray = gray[y:y+h, x:x+w]
        roi_color = img[y:y+h, x:x+w]
        
        eyes = eye_cascade.detectMultiScale(roi_gray)
        
        pupilFrame = roi_gray
        pupilO = roi_gray
        windowClose = np.ones((5,5),np.uint8)
        windowOpen = np.ones((2,2),np.uint8)
        windowErode = np.ones((2,2),np.uint8)
        
        for (ex,ey,ew,eh) in eyes:
            cv2.rectangle(roi_color,(ex,ey),(ex+ew,ey+eh),(0,255,0),2)
            cv2.line(roi_color, (ex,ey), ((ex+ew,ey+eh)), (0,0,255),1)
            cv2.line(roi_color, (ex+ew,ey), ((ex,ey+eh)), (0,0,255),1)
            pupilFrame = cv2.equalizeHist(roi_gray[int(ey+(eh*.25)):(ey+eh), ex:(ex+ew)])
            pupilO = pupilFrame
            ret, pupilFrame = cv2.threshold(pupilFrame,50,255,cv2.THRESH_BINARY)        #50 ..nothin 70 is better
            pupilFrame = cv2.morphologyEx(pupilFrame, cv2.MORPH_CLOSE, windowClose)
            pupilFrame = cv2.morphologyEx(pupilFrame, cv2.MORPH_ERODE, windowErode)
            pupilFrame = cv2.morphologyEx(pupilFrame, cv2.MORPH_OPEN, windowOpen)

            threshold = cv2.inRange(pupilFrame,250,255)        #get the blobs

            contours, hierarchy = cv2.findContours(threshold,cv2.RETR_LIST,cv2.CHAIN_APPROX_SIMPLE)

            if len(contours) >= 2:
                    #find biggest blob
                    maxArea = 0
                    MAindex = 0            #to get the unwanted frame 
                    distanceX = []        #delete the left most (for right eye)
                    currentIndex = 0 
                    for cnt in contours:
                        area = cv2.contourArea(cnt)
                        center = cv2.moments(cnt)
                        if center['m00'] != 0:
                            cx = int(center["m10"] / center["m00"])
                            cy = int(center["m01"] / center["m00"])
                        else:
                            cx,cy = 0, 0
                        distanceX.append(cx)    
                        if area > maxArea:
                            maxArea = area
                            MAindex = currentIndex
                        currentIndex = currentIndex + 1

                    del contours[MAindex]        #remove the picture frame contour
                    del distanceX[MAindex]

            eye = 'right'

            if len(contours) >= 2:        #delete the left most blob for right eye
                if eye == 'right':
                    edgeOfEye = distanceX.index(min(distanceX))
                else:
                    edgeOfEye = distanceX.index(max(distanceX))    
                del contours[edgeOfEye]
                del distanceX[edgeOfEye]

            if len(contours) >= 1:        #get largest blob
                    maxArea = 0
                    for cnt in contours:
                        area = cv2.contourArea(cnt)
                        if area > maxArea:
                            maxArea = area
                            largeBlob = cnt

            if len(largeBlob) > 0:    
                    center = cv2.moments(largeBlob)
                    cx,cy = int(center['m10']/center['m00']), int(center['m01']/center['m00'])
                    cv2.circle(pupilO,(cx,cy),5,255,-1)
            
            cv2.imshow('Eye',pupilO)
            cv2.imshow('Eye1',pupilFrame)
            
    cv2.imshow('img',img)
    k = cv2.waitKey(1) & 0xff
    if k == 27:
        break

cap.release()
cv2.destroyAllWindows()